# Exercise #2: Document-at-a-time scoring

Implement term-at-a-time scoring using vector space retrieval with TFIDF term weighting and dot product similarity.

In [1]:
from pprint import pprint
from collections import Counter
import math

#### Term-document matrix

In [2]:
td_matrix = {
    "beijing": [0, 1, 0, 0, 1],
    "dish": [0, 1, 0, 0, 1],
    "duck": [3, 2, 2, 0, 1],
    "rabbit": [0, 0, 1, 1, 0],
    "recipe": [0, 0, 1, 1, 1],
}

The number of documents is set manually for simplicity

In [3]:
NUM_DOCS = 5

#### Creating the corresponding inverted index

The postings hold (docID, freq) pairs. docID indices start from 0

`doclen` stores the document length

In [4]:
inv_idx = {}
doclen = {}
for term, vec in td_matrix.items():
    inv_idx[term] = []
    for doc_id, freq in enumerate(vec):
        if freq > 0:
            inv_idx[term].append((doc_id, freq))
            doclen[doc_id] = doclen.get(doc_id, 0) + freq

pprint(inv_idx)

{'beijing': [(1, 1), (4, 1)],
 'dish': [(1, 1), (4, 1)],
 'duck': [(0, 3), (1, 2), (2, 2), (4, 1)],
 'rabbit': [(2, 1), (3, 1)],
 'recipe': [(2, 1), (3, 1), (4, 1)]}


#### This class provides access to the inverted index

In [5]:
class InvIndex(object):
    def __init__(self, idx_contents):
        self.idx = idx_contents
    
    def postings(self, term):
        return self.idx.get(term, [])

Instantiate the InvIndex class

In [6]:
idx = InvIndex(inv_idx)

#### IDF calculation

In [7]:
def idf(term):
    return math.log(NUM_DOCS / len(idx.postings(term))) if len(idx.postings(term)) > 0 else 0

### Document-at-a-time scoring

We utilize the fact that the posting lists are ordered by document ID. 
The posting lists of the query terms are iterated parallel to each other.
Each document is scored according to

$score(q,d) = \sum_{t \in q} w_{t,d} \times w_{t,q}$

where $w_{t,d}=tf_{t,d}\times idf_t$ and $w_{t,q}=tf_{t,q}$

  - Use normalized frequencies for TF weight, i.e., $tf_{t,d}=\frac{f_{t,d}}{|d|}$, where $f_{t,d}$ is the number of occurrences of term $t$ in document $d$ and $|d|$ is the document length (=total number of terms). (It goes analogously for the query.)
  - Compute IDF values using the following formula: $idf_{t}=\log \frac{N}{n_t}$, where $N$ is the total number of document and $n_t$ is the number of documents that contain term $t$. 

(I.e., the same as in Exercise #1).

In [11]:
def score_dt(query, index):
    # Change the sequence of query terms into a "term: freq" dictionary
    qry = dict(Counter(query))

    scores = {}  # holds the final document scores (this should be a priority list, but for simplicity we use a dictionary here)
    
    # Iterate through each document
    for doc_id in range(NUM_DOCS):            
        # First, we collect the document term frequencies from the index
        # (Essentially, we just "recover" the document's contents from the index.)
        f_td = {}  # holds the term frequencies in the document
        for term in qry.keys(): 
            # TODO: get the frequency of query term i from the posting list
            # Utilize the fact that the posting lists are ordered by document ID!
            f_td[term] = 0
                    
        # Then, we score the document
        score = 0  # Holds the document's retrieval score
        for term, f_tq in qry.items():
            # Incement the document's score according to the given query term
            tfidf_td = 0  # TODO: compute the term's TFIDF score in the document
            tf_tq = f_tq / len(query)
            score += tfidf_td * tf_tq
        # Final document score
        scores[doc_id] = score
    return scores
                   

In [10]:
query = ["beijing", "duck", "recipe"]
scores = score_dt(query, idx)

In [11]:
for doc_id, score in sorted(scores.items(), key=lambda x: x[1], reverse=True):
    print("D" + str(doc_id + 1) + ":", round(score, 3))

D1: 0.0
D2: 0.0
D3: 0.0
D4: 0.0
D5: 0.0
